In [ ]:
I'll break down the exact steps to do this in Postman:

1. SETUP POSTMAN:

a. Install Postman
b. Create a GitHub Personal Access Token:
   - Go to GitHub.com → Settings → Developer Settings → Personal Access Tokens → Tokens (classic)
   - Generate new token
   - Select scopes: public_repo, read:user
   - Copy the token


2. CREATE ENVIRONMENT:

a. Click "Environments" in Postman sidebar
b. Click "+" to create new environment named "GitHub API"
c. Add these variables:
   - github_token: [your PAT token]
   - base_url: https://api.github.com
d. Click "Save"


3. CREATE COLLECTION:



markdown
# Request 1: Search Users
1. Create new collection "GitHub Delhi Users"
2. Add new request "Search Users"
   - Method: GET
   - URL: {{base_url}}/search/users
   - Query Params:
     - q: location:Delhi followers:>100
     - per_page: 30
     - page: 1
   - Headers:
     - Authorization: Bearer {{github_token}}
     - Accept: application/vnd.github.v3+json

# Request 2: Get User Details
1. Add new request "Get User Details"
   - Method: GET
   - URL: {{base_url}}/users/:username
   - Headers:
     - Authorization: Bearer {{github_token}}
     - Accept: application/vnd.github.v3+json

# Request 3: Get User Repositories
1. Add new request "Get User Repos"
   - Method: GET
   - URL: {{base_url}}/users/:username/repos
   - Query Params:
     - sort: pushed
     - per_page: 100
     - page: 1
   - Headers:
     - Authorization: Bearer {{github_token}}
     - Accept: application/vnd.github.v3+json

# Test Scripts for Each Request

## Search Users Test Script:
javascript
// Save user logins to environment
pm.test("Save user logins", function () {
    var jsonData = pm.response.json();
    var users = jsonData.items.map(item => item.login);
    pm.environment.set("user_logins", JSON.stringify(users));

    // Initialize CSV headers
    if (!pm.environment.get("users_csv")) {
        pm.environment.set("users_csv", "login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at");
    }
});


## User Details Test Script:
javascript
// Process and save user details
pm.test("Save user details", function () {
    var jsonData = pm.response.json();

    // Clean company name
    var company = jsonData.company ?
        jsonData.company.trim().replace(/^@/, '').toUpperCase() :
        "";

    // Create CSV row
    var csvRow = [
        jsonData.login,
        jsonData.name || "",
        company,
        jsonData.location || "",
        jsonData.email || "",
        jsonData.hireable || false,
        (jsonData.bio || "").replace(/,/g, ";"),
        jsonData.public_repos,
        jsonData.followers,
        jsonData.following,
        jsonData.created_at
    ].map(item => "${item}").join(',');

    // Append to CSV
    var existingCsv = pm.environment.get("users_csv");
    pm.environment.set("users_csv", existingCsv + "\n" + csvRow);
});


## Repositories Test Script:
javascript
// Process and save repository details
pm.test("Save repository details", function () {
    var jsonData = pm.response.json();
    var username = pm.request.url.path[1];  // Get username from URL

    // Initialize repos CSV if needed
    if (!pm.environment.get("repos_csv")) {
        pm.environment.set("repos_csv", "login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name");
    }

    // Process each repository
    jsonData.forEach(repo => {
        var csvRow = [
            username,
            repo.full_name,
            repo.created_at,
            repo.stargazers_count,
            repo.watchers_count,
            repo.language || "",
            repo.has_projects,
            repo.has_wiki,
            (repo.license && repo.license.key) || ""
        ].map(item => "${item}").join(',');

        // Append to CSV
        var existingCsv = pm.environment.get("repos_csv");
        pm.environment.set("repos_csv", existingCsv + "\n" + csvRow);
    });
});
```

```

4. RUN THE COLLECTION:

a. Run "Search Users" first
   - This gets list of all users
   - Check response has users

b. For each user from search:
   - Run "Get User Details"
   - Replace :username with user's login
   - Response saved automatically by test script

c. For each user:
   - Run "Get User Repos"
   - Replace :username with user's login
   - May need multiple pages (check response size)


5. EXPORT DATA:

a. Get users.csv:
   - View environment
   - Copy users_csv variable content
   - Paste into text editor
   - Save as users.csv

b. Get repositories.csv:
   - View environment
   - Copy repos_csv variable content
   - Paste into text editor
   - Save as repositories.csv


6. CREATE GITHUB REPOSITORY:

a. Go to GitHub.com
b. Click "+" → "New repository"
c. Name it "delhi-github-users"
d. Add files:
   - users.csv
   - repositories.csv
   - Create README.md


7. ANALYZE DATA AND UPDATE README:

a. Open CSVs in Excel/Google Sheets
b. Analyze:
   - Most common languages
   - Average followers
   - Popular licenses
   - Company distribution

c. Write README.md with:
   - Scraping explanation
   - Interesting finding
   - Developer recommendation


Would you like me to:
1. Show example analysis queries?
2. Provide more detailed test scripts?
3. Help with any specific part of the process?

Let me know what part needs more clarification!
import pandas as pd
import statsmodels.api as sm

# Load the CSV file
csv_file = 'users.csv'  # Ensure this path is correct

# Load the CSV into a DataFrame
df = pd.read_csv(csv_file)

# Check the first few rows and the data types of the DataFrame
print("DataFrame Overview:")
print(df.head())
print("\nDataFrame Info:")
print(df.info())

# Filter out users without bios
df = df[df['bio'].notnull()]

# Calculate the length of each bio in words
df['bio_word_count'] = df['bio'].str.split().str.len()

# Prepare the independent variable (X) and dependent variable (y)
X = df['bio_word_count']
y = df['followers']  # Adjust the column name as per your dataset

# Add a constant to the independent variable (for the intercept)
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Get the slope (coefficient of the bio_word_count)
slope = model.params['bio_word_count']

# Print the regression slope rounded to three decimal places
print(f"\nRegression slope of followers on bio word count: {slope:.3f}")